In [1]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorboardX import SummaryWriter
import time
import numpy as np
from PIL import Image

In [2]:
# Configuration Variables
MEAN_PREPRO = [0.485, 0.456, 0.406]
STD_PREPRO = [0.229, 0.224, 0.225]
RESIZE_PREPRO = 256,256
RESIZE_DRAW = 256,256

TRAIN_BATCH_SIZE = 128
TRAIN_SHUFFLE = True
TRAIN_NUM_WORKERS = 8
TRAIN_PIN_MEMORY = True

VAL_BATCH_SIZE = 512
VAL_SHUFFLE = False
VAL_NUM_WORKERS = 4
VAL_PIN_MEMORY = True

INITIAL_LR = 1e-4
DEVICE_ID = 1

DATA_DIR = '/data/porn/binary/'
TRAINSET_ROOT = f'{DATA_DIR}/train'
TESTSET_ROOT = f'{DATA_DIR}/test'

TENSORBOARD_DIR = '/data/porn/tensorboard/resnext101/'

In [3]:
normalize = transforms.Normalize(
        mean=MEAN_PREPRO, std=STD_PREPRO
    )
prepro = transforms.Compose(
    [
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]
)
prepro_val = transforms.Compose(
    [transforms.Resize(RESIZE_PREPRO), transforms.ToTensor(), normalize]
)

prepro_draw = transforms.Compose(
    [transforms.Resize(RESIZE_DRAW), transforms.ToTensor()]
)


In [4]:
class validation_dataset(ImageFolder):
    def __init__(self, root, transform):
        super().__init__(root, transform)
        
    def __getitem__(self, index):
        data, label = super().__getitem__(index)
        return data, label, index

In [ ]:
train_dataset = ImageFolder(TRAINSET_ROOT, transform=prepro)
val_dataset = validation_dataset(TESTSET_ROOT, transform=prepro_val)
print(train_dataset.classes)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=TRAIN_SHUFFLE, num_workers=TRAIN_NUM_WORKERS, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=VAL_SHUFFLE, num_workers=VAL_NUM_WORKERS, pin_memory=VAL_PIN_MEMORY, drop_last=False)

In [7]:
model = models.resnext101_32x8d(pretrained=True)
model.fc = torch.nn.Linear(2048, len(train_dataset.classes))
DEVICE = f"cuda:{DEVICE_ID}"
model = model.to(DEVICE)
for p in model.parameters():
    p.requires_grad=False
for p in model.fc.parameters():
    p.requires_grad=True

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/checkpoints/resnext101_32x8d-8ba56ff5.pth


In [29]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [8]:
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR)
criterion = nn.CrossEntropyLoss()

In [9]:
def plot_cm(classes, cm):
    fig, ax = plt.subplots(figsize=(len(classes),len(classes)))  
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax,fmt="d")
    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels') 
    ax.set_title('Confusion Matrix') 
    ax.xaxis.set_ticklabels(classes,rotation=90)
    ax.yaxis.set_ticklabels(classes,rotation=0)
    return fig

In [10]:
def train(
    train_loader,
    model,
    criterion,
    optimizer,
    epoch,
    on_iteration=None,
):
    model = model.train()
    end = time.time()
    print("Start Training")
    avg_loss = 0
    for i, (inputs, labels) in enumerate(train_loader):
        print(f"{i/len(train_loader) * 100 : 2.2f}%", end="\r")
        iteration_time = time.time()
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        avg_loss += loss.item()
        loss.backward()
        optimizer.step()
        if on_iteration is not None:
            on_iteration(iteration=i+epoch*len(train_loader), loss=loss, y_pred=outputs, y_true=labels)     
    return avg_loss/len(train_loader)

In [11]:
def validate(val_loader, 
             model, 
             criterion,
             print_freq=1000):
    model = model.eval()
    y_pred, y_true, proba_pred, indexes = [], [], [], []
    avg_loss = 0
    for i, (inputs, labels, image_indexes) in enumerate(val_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        with torch.no_grad():
            outputs = model(inputs)
            avg_loss += criterion(outputs, labels).item()
        proba_pred.append(outputs.cpu().clone())
        _, indices_max = torch.max(outputs.cpu(), 1)
        y_pred.append(indices_max)
        y_true.append(labels.cpu().clone())
        indexes.append(image_indexes.clone())
    return {"loss":avg_loss/len(val_loader), 
            "class_prediction":torch.cat(y_pred), 
            "ground_truth":torch.cat(y_true), 
            "probabilities":torch.cat(proba_pred), 
            "images_index":torch.LongTensor(torch.cat(indexes))}

In [51]:
def draw_max_error(epoch, val_res, n=5):
    # max false positive
    fp = (val_res["class_prediction"] == 0) & (val_res["ground_truth"] == 1)
    values, images = [], []
    for i in range(len(fp)):
        if fp[i]:
            values.append(val_res["probabilities"][i][1])
            images.append(val_dataset.imgs[val_res["images_index"][i]][0])
    _, max_indexes = torch.sort(torch.Tensor(values), descending=True)
    for i in range(n):
        logger.add_image(f"Eval/FalsePositive/{i}", prepro_draw(Image.open(images[max_indexes[i]])), epoch)
        
    #false negative
    fn = (val_res["class_prediction"] == 1) & (val_res["ground_truth"] == 0)
    values, images = [], []
    for i in range(len(fn)):
        if fn[i]:
            values.append(val_res["probabilities"][i][0])
            images.append(val_dataset.imgs[val_res["images_index"][i]][0])
    _, max_indexes = torch.sort(torch.Tensor(values), descending=True)
    for i in range(n):
        logger.add_image(f"Eval/FalseNegative/{i}", prepro_draw(Image.open(images[max_indexes[i]])), epoch)

In [52]:
def compute_score(val_res):
    VP = (val_res["class_prediction"] == 0) & (val_res["ground_truth"] == 0)
    FP = (val_res["class_prediction"] == 0) & (val_res["ground_truth"] == 1)
    FN = (val_res["class_prediction"] == 1) & (val_res["ground_truth"] == 0)
    vp = float(VP.sum())
    fp = FP.sum()
    fn = FN.sum()
    precision = vp / (vp + fp)
    recall = vp / (vp + fn)
    return precision, recall

In [53]:
def validation_logs(epoch, validation_res):
    logger.add_scalar("Loss/Avg_Val", validation_res["loss"], epoch)
    logger.add_scalar("Eval/Accuracy", (validation_res["class_prediction"]==validation_res["ground_truth"]).sum()/float(len(validation_res["ground_truth"])), epoch)
    precision, recall = compute_score(validation_res)
    logger.add_scalar("Eval/Recall", recall, epoch)
    logger.add_scalar("Eval/Precision", precision, epoch)
    probas = [x[validation_res["ground_truth"][i]] for i,x in enumerate(validation_res["probabilities"])]
    logger.add_pr_curve("Eval/Prec_recall", validation_res["ground_truth"], probas, epoch)
    draw_max_error(epoch, validation_res)
    cm = confusion_matrix(validation_res["class_prediction"], validation_res["ground_truth"])
    cm_image = plot_cm(train_dataset.classes, cm)
    logger.add_figure("Eval/Confusion Matrix", cm_image, epoch)

In [54]:
def on_iteration_logs(iteration, loss, y_pred, y_true):
    l = loss.item()
    if iteration%50 == 0:
        logger.add_scalar("Loss/Train", l, iteration)
        print(
                f"{iteration}/{len(train_loader)} \t"
                f"Loss {l}"
            )

In [55]:
logger = SummaryWriter(TENSORBOARD_DIR)

In [ ]:
for i in range(10):
    val_res = validate(val_loader, model, criterion)
    validation_logs(i, val_res)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)

In [17]:
for i, child in enumerate(model.children()):
    if i > 6:
        for p in child.parameters():
            p.requires_grad=True

In [18]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE//2, shuffle=TRAIN_SHUFFLE, num_workers=TRAIN_NUM_WORKERS, pin_memory=True, drop_last=True)

In [19]:
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR*0.1)

In [ ]:
for i in range(10,40):
    val_res = validate(val_loader, model, criterion)
    validation_logs(i, val_res)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)

Start Training
28650/2862 	Loss 0.1686488538980484
28700/2862 	Loss 0.2095438539981842
28750/2862 	Loss 0.23723465204238892
28800/2862 	Loss 0.24509967863559723
28850/2862 	Loss 0.12935705482959747
28900/2862 	Loss 0.22119750082492828
28950/2862 	Loss 0.2232341766357422
29000/2862 	Loss 0.1304795742034912
29050/2862 	Loss 0.18364578485488892
29100/2862 	Loss 0.2123449593782425
29150/2862 	Loss 0.21678024530410767
29200/2862 	Loss 0.15836134552955627
29250/2862 	Loss 0.16371679306030273
29300/2862 	Loss 0.15101487934589386
29350/2862 	Loss 0.23681676387786865
29400/2862 	Loss 0.22939768433570862
29450/2862 	Loss 0.22951573133468628
29500/2862 	Loss 0.20864397287368774
29550/2862 	Loss 0.11768437176942825
29600/2862 	Loss 0.17191599309444427
29650/2862 	Loss 0.12864845991134644
29700/2862 	Loss 0.17533104121685028
29750/2862 	Loss 0.17011667788028717
29800/2862 	Loss 0.074493907392025
29850/2862 	Loss 0.15106551349163055
29900/2862 	Loss 0.10724467039108276
29950/2862 	Loss 0.18552917242

In [ ]:
for i, child in enumerate(model.children()):
    if i > 3:
        for p in child.parameters():
            p.requires_grad=True
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR*0.01)

In [ ]:
for i in range(40,80):
    val_res = validate(val_loader, model, criterion)
    validation_logs(i, val_res)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)